# Library

In [1]:
#import sys
import pandas as pd
import numpy as np
import random
import pyspark
import itertools
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import pickle
import statsmodels.api as sm

from pyspark import SparkContext, SQLContext

from math import sqrt
from time import time as ttt

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier as DTC_spark
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import joblib
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import parallel_backend


# Upload

In [2]:
spark = SparkSession.builder.master("Yarn").appName("spark_app_1234").getOrCreate()
sc = spark.sparkContext
spark
d0 = (
    spark
    .read
    .format("csv") 
    .option("header","true") 
    .option("inferSchema","true") 
    .load("gs://mas-a5-storage-1/notebooks/jupyter/application_train.csv")
)

d1 = d0.filter(d0.DAYS_EMPLOYED != 365243).select('TARGET','DAYS_EMPLOYED')

print(d1.rdd.getNumPartitions())

2


In [3]:
sc

<SparkContext master=yarn appName=PySparkShell>

# Function

In [4]:
def prepare_spark_data(n_part, k_mult):
    '''
    takes 'DAYS_EMPLOYED and 'TARGET'  from d1 (alreay filtered)
    rearrane in n partitions (if n==0 keeps initial number of partitions)
    prints final shape/ partition
    returns d2 - spark df
    '''
    data = d1
    data_new = data # first step in the cycle
    for i in range(k_mult-1):
        data_tmp = data.select('TARGET', \
                     f.col('DAYS_EMPLOYED')*(f.lit(0.9995) + f.rand()/1000)).\
                   toDF('TARGET','DAYS_EMPLOYED')
        data_tmp = data_tmp.select('TARGET', f.floor('DAYS_EMPLOYED'))
        data_new = data_new.union(data_tmp)

    assembler = VectorAssembler(inputCols=["DAYS_EMPLOYED"], 
                        outputCol="DAYS_EMPLOYED_vect")
    d2 = assembler.transform(data_new)
    if n_part != 0:
        d2 = d2.repartition(n_part)       
    #print(f'n-partitions initial: {d2.rdd.getNumPartitions()}; df size: {d2.count()}\n')

    return d2

# Params an run

In [ ]:
## !! mind the fn NOT TO rewrite results
fn = '4cpu_by_8n__4m_yarn_ssd'
size_mult = [1, 2, 5, 10, 15, 20, 30, 50, 100]
partitions = [2, 4, 8, 16, 24] 
# initial, 2**i incl n_nodes till n_cpu*n_nodes and n_cpu*n_nodes
n_iter = 3

print('d1-size', d1.count())
n_part_base = d1.rdd.getNumPartitions()
print('n-partitions initial', n_part_base, '\n')

rd1 = {}
for k_size_mult in size_mult:
    rd2 = {}
    for n_part in partitions:
        df = prepare_spark_data(n_part, k_size_mult)
        df.cache()
        print('======================================================================')
        print(f'=== size_mult={k_size_mult}; \
        n_part_req={n_part}; n_part_act={df.rdd.getNumPartitions()}; df_count={df.count()} ===\n')
        times = [0 for i in range(n_iter)]
        models = {}
        for i in range(n_iter):
            dt = DTC_spark(labelCol="TARGET",
                       featuresCol="DAYS_EMPLOYED_vect",
                       minInfoGain=0.0001,
                       impurity='entropy',
                       maxDepth=14, maxBins=2**14, # it differs from scikit learn - it means number of canidate split points
                       #minInstancesPerNode = 1,
                       #checkpointInterval = 10
                       )
           
            t0 = ttt()
            model = dt.fit(df)
            t1 = ttt()
            times[i] = t1-t0
            models[i] = model 
            print('model', i, 'build time', round(times[i],2), '\n', model)
        rd2[n_part] = (models, times)
    rd1[k_size_mult] = rd2

result = rd1  

d1-size 252137
n-partitions initial 2 



=== size_mult=1;         n_part_req=2; n_part_act=2; df_count=252137 ===



model 0 build time 10.36 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_4bf3e1464934, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 1 build time 7.47 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_787e14ea66a0, depth=14, numNodes=123, numClasses=2, numFeatures=1


model 2 build time 7.0 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_2effd517921b, depth=14, numNodes=123, numClasses=2, numFeatures=1


...

model 2 build time 21.71 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_a3e676d26e46, depth=12, numNodes=43, numClasses=2, numFeatures=1


=== size_mult=100;         n_part_req=24; n_part_act=24; df_count=25213700 ===



model 0 build time 25.55 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_7df6102373a6, depth=11, numNodes=43, numClasses=2, numFeatures=1


model 1 build time 22.32 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_f391e6126ec4, depth=11, numNodes=43, numClasses=2, numFeatures=1


model 2 build time 22.54 
 DecisionTreeClassificationModel: uid=DecisionTreeClassifier_4dc2a955b56c, depth=11, numNodes=43, numClasses=2, numFeatures=1


# Results

In [ ]:
di = result
df_res_time = pd.DataFrame()
df_res_nodes = pd.DataFrame()
for p1 in di.keys():
    for p2 in di[p1].keys():
        df_res_time.loc[p1, p2] = np.round(np.median(di[p1][p2][1]), 1)
        m_tmp = di[p1][p2][0]
        n_nodes = []
        for i,_ in enumerate(m_tmp):
            tmp = f'{m_tmp[i]}'.split(' ')
            print(i, tmp)
            n_nodes.append([int(x.split('=')[1][:-1]) for x in tmp if x[:4]== 'numN'][0])
        df_res_nodes.loc[p1, p2] = np.round(np.mean(n_nodes), 1)

df_res_time.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
df_res_nodes.to_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')

display(df_res_time)      
display(df_res_nodes)     

0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_4bf3e1464934,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_787e14ea66a0,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_2effd517921b,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_418ed98697d0,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
1 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_fdff04196ea3,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
2 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_404d5c7810f2,', 'depth=14,', 'numNodes=123,', 'numClasses=2,', 'numFeatures=1']
0 ['DecisionTreeClassificationModel:', 'uid=DecisionTreeClassifier_14f67b7fffd1,', 'depth=14,', 'numNodes=123,', 'numC

,2,4,8,16,24
1,7.5,8.3,10.6,15.1,16.9
2,10.6,7.3,6.3,8.8,10.7
5,12.7,10.3,9.9,24.5,10.2
10,17.5,13.5,9.8,7.2,8.1
15,23.5,15.7,10.5,7.4,9.0
20,30.8,16.8,11.7,8.6,10.1
30,38.6,21.8,14.6,9.7,11.6
50,53.5,33.7,20.8,13.9,14.4
100,98.1,58.9,39.3,21.7,22.5


,2,4,8,16,24
1,123.0,123.0,123.0,123.0,123.0
2,101.0,131.0,135.0,151.0,153.0
5,83.0,113.0,97.0,109.0,79.0
10,83.0,105.0,81.0,71.0,89.0
15,51.0,63.0,59.0,59.0,79.0
20,63.0,57.0,57.0,31.0,59.0
30,41.0,43.0,55.0,43.0,31.0
50,41.0,39.0,45.0,35.0,27.0
100,39.0,33.0,45.0,43.0,43.0


In [7]:
#fn = '4cpu_by_8n__4m_yarn_ssd'
tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_t.csv')
print(fn)
tmp

4cpu_by_8n__4m_yarn_ssd


,Unnamed: 0,2,4,8,16,24
0,1,7.5,8.3,10.6,15.1,16.9
1,2,10.6,7.3,6.3,8.8,10.7
2,5,12.7,10.3,9.9,24.5,10.2
3,10,17.5,13.5,9.8,7.2,8.1
4,15,23.5,15.7,10.5,7.4,9.0
5,20,30.8,16.8,11.7,8.6,10.1
6,30,38.6,21.8,14.6,9.7,11.6
7,50,53.5,33.7,20.8,13.9,14.4
8,100,98.1,58.9,39.3,21.7,22.5


In [8]:
#fn = '4cpu_by_8n__4m_yarn_ssd'
tmp = pd.read_csv(f'gs://mas-a5-storage-1/notebooks/jupyter/obj/{fn}_n.csv')
print(fn)
tmp

4cpu_by_8n__4m_yarn_ssd


,Unnamed: 0,2,4,8,16,24
0,1,123.0,123.0,123.0,123.0,123.0
1,2,101.0,131.0,135.0,151.0,153.0
2,5,83.0,113.0,97.0,109.0,79.0
3,10,83.0,105.0,81.0,71.0,89.0
4,15,51.0,63.0,59.0,59.0,79.0
5,20,63.0,57.0,57.0,31.0,59.0
6,30,41.0,43.0,55.0,43.0,31.0
7,50,41.0,39.0,45.0,35.0,27.0
8,100,39.0,33.0,45.0,43.0,43.0
